В этом ноутбуке кортежи мнений предсказываются с помощью модели Mistral Large 2 (mistral-large-latest).

Для использования необходим Mistral API key, который можно получить бесплатно.

Для формирования затравок используются предсказания других больших языковых моделей (см. ноутбуки gpt4 и grok в этом репозитории). Они берутся из .csv файлов, которых нет в открытом доступе.

Без этих файлов воспроизвести происходящие в этом ноутбуке не получится.

### Проверка ###

In [23]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

api_key = "YOUR_API_KEY"
model = "mistral-large-2411"
url = "https://api.mistral.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
SEED = 42

random.seed(SEED)

In [24]:
response = requests.get(url)
response.headers

{'Date': 'Sun, 24 Nov 2024 23:25:07 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '96', 'Connection': 'keep-alive', 'www-authenticate': 'Key', 'access-control-allow-origin': '*', 'x-kong-response-latency': '0', 'x-kong-request-id': '8cffbf14767253180480730780fa685f', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '8e7d280bc9e5d68f-IAD', 'alt-svc': 'h3=":443"; ma=86400'}

In [25]:
class MistralModel():
    def __init__(self, model, temp = 0.2, top_p = 0.9, max_tokens = 256, seed = 42):
        self.model = model
        self.temp = temp
        self.top_p = top_p
        self.max_tokens = max_tokens
        self.seed = 42
    def prompt(self, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'user', 'content': text}]
             }
        return prompt
    
    def structured_prompt(self, instruction, text):
        prompt = {'model': self.model,
                  'random_seed': self.seed,
              'temperature': self.temp,
              'top_p': self.top_p,
              'max_tokens': self.max_tokens,
              'messages': [{'role':'system', 'content': instruction},
                          {'role':'user', 'content': text}]
             }
        return prompt

mistral = MistralModel(model, max_tokens = 512)

instruction = "ответь на вопрос"
text = "тебя создали французы?"
response = requests.post(url, headers=headers, json=mistral.prompt(text))
print(f'Вопрос: {text}')
print(f"Ответ: {response.json()['choices'][0]['message']['content']}")

Вопрос: тебя создали французы?
Ответ: Нет, меня создала Mistral AI, передовая французская компания в области искусственного интеллекта.


### Подготовка данных ###

In [6]:
import requests, zipfile, io

# download source files
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/src/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

In [7]:
from src.utils import load_jsonl, save_jsonl, str2list, dict2tuple, extract_tuple, df2structure

In [8]:
train_path = "full.jsonl"
test_path = "validation.jsonl"

url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/data/full.jsonl'
train = load_jsonl(url, train_path)
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/data/validation.jsonl'
test = load_jsonl(url, test_path)

print(len(train), len(test))

2556 1316


In [9]:
# mistral = pd.read_csv('/content/gdrive/My Drive/Mistral_bl_15shot_0.1temp.csv')
# grok = pd.read_csv('/content/gdrive/My Drive/Grok_bl_15shot_0.1temp.csv')
# claude = pd.read_csv('/content/gdrive/My Drive/Claude_bl_15shot_0.1temp.csv')
grok = pd.read_csv('results/Grok/Grok_bl_15shot_0.1temp.csv')
openai = pd.read_csv('results/openai/openai_bl_15shot_0.1temp.csv')
claude = pd.read_csv('results/Claude/Claude_bl_15shot_0.1temp.csv')

aug = [grok, openai, claude]

In [10]:
claude.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],[]
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[['AUTHOR', 'они', 'снимали нас все эти 5 мину..."
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[['Светлана', 'сына', 'синяк', 'NEG']]"


In [12]:
claude.loc[3]['pred']

"[['AUTHOR', 'они', 'снимали нас все эти 5 минут, что длилось свидание, чтобы Влад ничего лишнего мне не сказал', 'NEG']]"

### Определения ###

In [15]:
def aug_prompt(examples, variants, text):
    shots = '\n'.join([f'Текст: {pair[0]}\nОтвет: {pair[1]}' for pair in examples])
    variants = '\n'.join([str(x) for x in variants])
    return f"""Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
{shots}
Текст, который нужно проанализировать:
{text}
Ответы экспертов к этому тексту:
{variants}
Ты можешь выбрать из этих ответов или ответить по-своему.
Твой ответ:"""

class Runner():
    def __init__(self, model, url, headers, train, test, aug, n_shots = 5, sleeptime = 2):
        self.model = model
        self.url = url
        self.headers = headers
        self.train = train
        self.test = test
        self.n_shots = n_shots
        self.sleeptime = sleeptime
        self.aug = aug
    
    def run(self):
        results = list()
        for i in tqdm(range(len(self.test))):
            entry = self.test[i]
            time.sleep(self.sleeptime)
            examples = [dict2tuple(x) for x in choices(self.train, k = n_shots)]
            variants = [x.loc[i]['pred'] for x in self.aug]
            prompt = aug_prompt(examples, variants, entry['text'])
            response = requests.post(self.url,
                                     headers=self.headers,
                                     json=self.model.prompt(prompt))
            result = []
            if response.status_code == 200:
                response = response.json()['choices'][0]['message']['content']
                try:
                    result = ast.literal_eval(response)
                except (SyntaxError, ValueError):
                    print(f'bad response, iteration:{len(results)}')
            else:
                print(f'bad response, iteration:{len(results)}') 
            results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return results

def get_path(temp, n_shots):
    path = f'./results/Mistral-aug/Mistral_bl_{n_shots}shot_{temp}temp'
    # returns full path but without ".csv"
    return path

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

### Тест на одном примере ###

In [69]:
n_shots = 0
examples = [dict2tuple(x) for x in train[:n_shots]]
text, target = dict2tuple(test[14])
variants = [x.loc[14]['pred'] for x in aug]

sample_prompt = aug_prompt(examples, variants, text)
print(f'--Промпт--:\n\n{sample_prompt}')

--Промпт--:

Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры

Текст, который нужно проанализировать:
Все полученные средства планируется выплатить акционерам Yahoo!
Ответы экспертов к этому тексту:
[]
[]
[['NULL', 'акционерам Yahoo!', 'планируется выплатить', 'POS']]
Ты можешь выбрать из этих ответов или ответить по-своему.
Твой ответ:


In [27]:
response = requests.post(url, headers=headers, json=mistral.prompt(sample_prompt))
response = response.json()['choices'][0]['message']['content']
result = ast.literal_eval(response)

print(f'--Текст--:\n{text}')
print(f'--Таргет--:\n{target}')
print(f'--Предикт--:\n{extract_tuple(result)}')

--Текст--:
Напомним, на казахстанском пограничном временном посту "Арканкерген" были обнаружены обгоревшие тела 14 пограничников и 1 егеря.
--Таргет--:
[]
--Предикт--:
[['NULL', 'пограничников и егеря', 'обгоревшие тела', 'NEG']]


### Инференс ###

In [32]:
%%time
params = {'temp': 0.1,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}
mistral = MistralModel(model, **params)
n_shots = 12
runner = Runner(mistral, url, headers, train, test, aug, n_shots)

path = get_path(mistral.temp, n_shots)
result = runner.run()

 12%|█▏        | 159/1316 [09:55<1:34:02,  4.88s/it]

bad response, iteration:158


 13%|█▎        | 165/1316 [10:17<1:12:13,  3.76s/it]

bad response, iteration:164


 15%|█▍        | 191/1316 [12:37<1:29:32,  4.78s/it]

...

bad response, iteration:1016


 84%|████████▍ | 1108/1316 [1:18:31<16:28,  4.75s/it]

bad response, iteration:1107


100%|██████████| 1316/1316 [1:32:35<00:00,  4.22s/it]

CPU times: total: 7min 16s
Wall time: 1h 32min 35s


In [34]:
print(path)
print(result[4])

./results/Mistral-aug/Mistral_bl_12shot_0.1temp
(4, 'Кроме того, по словам женщины на щеке сына Светлана увидела синяк.', [], [['Светлана', 'сына', 'синяк', 'NEG']])


### Результаты ###

In [35]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],[]
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[[AUTHOR, они, снимали нас все эти 5 минут, чт..."
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[Светлана, сына, синяк, NEG]]"


In [36]:
print(path)
save(output, path)

./results/Mistral-aug/Mistral_bl_12shot_0.1temp


In [37]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],[]
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[[AUTHOR, они, снимали нас все эти 5 минут, чт..."
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[Светлана, сына, синяк, NEG]]"


In [38]:
save(output, path, raw = False)

In [13]:
# short_report(output)

Count: 176
Accuracy: 0.344
NaNs: 0


### csv to jsonl ###

In [39]:
final = df2structure(output)
final[3]

{'sent_id': 3,
 'text': 'Они снимали нас все эти 5 минут, что длилось свидание, чтобы Влад ничего лишнего мне не сказал.',
 'opinions': [{'Source': [['AUTHOR'], ['NULL']],
   'Target': [['они'], ['0:0']],
   'Polar_expression': [['снимали нас все эти 5 минут, что длилось свидание, чтобы Влад ничего лишнего мне не сказал'],
    ['4:94']],
   'Polarity': 'NEG'}]}

In [44]:
def to_jsonl(data, target):
    with open(target, "w", encoding='utf8') as f:
        for item in data:
            f.write(f"{json.dumps(item, ensure_ascii=False)}\n")

def save_jsonl(dataframe, path):
    path+='.jsonl'
    return to_jsonl(dataframe, path)

save_jsonl(final, path)